In [22]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import line_search

In [23]:
def rosenbrock(x:np.ndarray):
#   Define the object function
    return 100.0 * (x[1] - x[0]**2)**2 + (1.0 - x[0])**2

In [24]:
def grad_rosenbrock(x:np.ndarray):
#   Gradient of the Rosenbrock function.

    x_val, y_val = x[0], x[1]
    df_dx = -400.0 * x_val * (y_val - x_val**2) - 2.0 * (1.0 - x_val)
    df_dy =  200.0 * (y_val - x_val**2)
    return np.array([df_dx, df_dy])

In [25]:
def h(x):

    return x[0] + x[1] - 1.0

def grad_h(x):

    return np.array([1.0, 1.0])

In [26]:
def aug_lagrangian(x, lam, rho):

    return rosenbrock(x) + lam*h(x) + 0.5*rho*(h(x)**2)

def grad_aug_lagrangian(x, lam, rho):

    return grad_rosenbrock(x) + lam*grad_h(x) + rho*h(x)*grad_h(x)

In [27]:
def backtracking_line_search(x, p, grad, func, c=1e-4, rho=0.8):

    alpha = 1.0
    fx = func(x)
    while True:
        x_new = x + alpha * p
        lhs = func(x_new)
        rhs = fx + c * alpha * np.dot(grad, p)
        if lhs <= rhs:
            break
        alpha *= rho
        if alpha < 1e-16:
            break
    return alpha

In [28]:
def bfgs(x0:np.ndarray, lam, rho, tol=1e-7, max_iter=100000):

    n = len(x0)
    x = x0
    B = np.eye(n)  # initial inverse Hessian surrogate
    iter = 0

    while iter < max_iter:
        grad_L = grad_aug_lagrangian(x, lam, rho)
        if np.linalg.norm(grad_L) < tol:
            break
        
        p = -np.linalg.solve(B, grad_L) 
        alpha = backtracking_line_search(x, p, grad_L,
                                         lambda z: aug_lagrangian(z, lam, rho))

        s = alpha * p
        x_new = x + s
        grad_L_new = grad_aug_lagrangian(x_new, lam, rho)
        y = grad_L_new - grad_L
        # Curvature condition
        if np.dot(y, s) > 1e-10:
            d1 = np.inner(s, B @ s)
            d2 = np.inner(y, s)
            B = B - (1/d1) * np.outer(B@s, s@B) + (1/d2) * np.outer(y, y)

        x = x_new
        iter += 1
        
    return x, iter

In [29]:
def augmented_lagrangian_method(x0, tol=1e-6, outer_it=20):
    """
    Solve the constrained problem:
        min f(x)
        s.t. g(x) = 0
    using the Augmented Lagrangian approach:
      L(x; lam, rho) = f(x) + lam*g(x) + (rho/2)*[g(x)]^2.
    Use BFGS to solve each subproblem.
    """
    # Initialize Lagrange rholtiplier and penalty
    xk = x0
    lam = 0.0
    rho = 1.0
    
    total_iter = 0
    for outer_iter in range(outer_it):
    
        xk, bfgs_its = bfgs(xk, lam, rho, tol=tol)
        total_iter += bfgs_its
        
        # primal feasibility
        primal_resid = abs(h(xk))    
        # Since there is no inequality constraints, no need of dual feasibility
        
        if primal_resid < tol:
            break
        
        # Standard augmented Lagrangian update
        lam = lam + rho*h(xk)
        
        # Increase penalty term by \gamma=10
        if primal_resid > tol:
            rho *= 10.0
    
    return xk, lam, outer_iter+1, total_iter

In [31]:
x0 = np.array([-1.0, 1.0])
    
# Solve with Augmented Lagrangian
x_opt, lam_opt, outer_iters, total_bfgs_its = augmented_lagrangian_method(x0, tol=1e-6)

# Print results
print("Augmented Lagrangian Method Results")
print("------------------------------------")
print(f"Starting point:        {x0}")
print(f"Optimal solution x*:   {x_opt}")
print(f"Optimal Lagrange λ*:   {lam_opt:.6f}")
print(f"Outer iterations:      {outer_iters}")
print(f"Total BFGS iterations: {total_bfgs_its}")
print(f"Constraint value h(x*)= {h(x_opt):.6e}")
print(f"Final objective f(x*) = {rosenbrock(x_opt):.6e}")

Augmented Lagrangian Method Results
------------------------------------
Starting point:        [-1.  1.]
Optimal solution x*:   [0.61879563 0.3812044 ]
Optimal Lagrange λ*:   0.340698
Outer iterations:      4
Total BFGS iterations: 53
Constraint value h(x*)= 2.916340e-08
Final objective f(x*) = 1.456070e-01


The goal of increasing penalty term is to approximate $\rho \to \inf $, so that solution approach feasible region.